In [ ]:
# 2020-10-22 created by Akson

In [ ]:
# Code4.1
# 生成一些测试数据

import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [ ]:
# Code4.2
# 画一下

import matplotlib.pyplot as plt

plt.scatter(X, y)
plt.axis([0, 2, 0, 15])
plt.show()

In [ ]:
# Code4.3
# 使用标准方程计算

X_b = np.c_[np.ones((100, 1)), X]
# print(X_b)

theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print(theta_best)

In [ ]:
# Code4.4
# 预测新值

X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
y_predict = X_new_b.dot(theta_best)
print(y_predict)

In [ ]:
# Code4.5
# 画一下

plt.plot(X_new, y_predict, 'r-')
plt.plot(X, y, 'b.')
plt.axis([0, 2, 0, 15])
plt.show()

In [ ]:
# Code4.6
# 使用sklearn的线性回归模型

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)

print(lin_reg.intercept_)
print(lin_reg.coef_)

predictions = lin_reg.predict(X_new)
print(predictions)

In [ ]:
# Code4.7
# sklearn使用的是这个最小二乘法函数，我们也可以直接调用它

theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond = 1e-6)

print(theta_best_svd)

# 也可以掉用计算伪逆这个函数
print(np.linalg.pinv(X_b).dot(y))

In [ ]:
# Code4.8
# 手写批量梯度下降算法

eta = 0.1
n_iteration = 1000
m = 100

theta = np.random.randn(2, 1)

for iteration in range(n_iteration):
    gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients
    
print(theta)

In [ ]:
# Code4.9
# 手写随机梯度下降与模拟退火

theta_path_sgd = []
m = len(X_b)

n_epochs = 50
t0, t1 = 5, 50

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2, 1)

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:
            y_predict = X_new_b.dot(theta)
            style = 'b-' if i > 0 else 'r--'
            plt.plot(X_new, y_predict, style)

        random_index = np.random.randint(m)
        xi = X_b[random_index : random_index + 1]
        yi = y[random_index: random_index + 1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)
        
plt.plot(X, y, 'b.')
plt.xlabel('$x_1$', fontsize = 18)
plt.ylabel('$y$', rotation = 0, fontsize = 18)
plt.axis([0, 2, 0, 15])
plt.show()

In [ ]:
# Code4.10
# 使用sklearn中的SGDRegressor

from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter = 1000, tol = 1e-3, penalty = None, eta0 = 0.1)
sgd_reg.fit(X, y.ravel())

print(sgd_reg.intercept_)
print(sgd_reg.coef_)

In [ ]:
# Code4.(10.5)
# 小批量梯度下降(2021-01-17)

theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2, 1)

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i: i + minibatch_size]
        yi = y_shuffled[i: i + minibatch_size]
        gradients = 2 / minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

print(theta)

In [ ]:
# Code4.11
# 为多项式回归生成数据集

m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X ** 2 + np.random.randn(m, 1)

plt.scatter(X, y)
plt.axis([-3, 3, -5, 10])
plt.show()

In [ ]:
# Code4.12
# 使用多项式回归来拟合

from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree = 2, include_bias = False)
X_poly = poly_features.fit_transform(X)

print(X[0])
print(X_poly[0])

lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

print(lin_reg.intercept_)
print(lin_reg.coef_)

In [ ]:
# Code4.13
# 绘制学习曲线，欠拟合

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
    train_errors = []
    val_errors = []
    for m in range(1, len(X_train)):
        model.fit(X_train[: m], y_train[: m])
        y_train_predict = model.predict(X_train[: m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[: m ], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), 'r-+', linewidth = 2, label = 'train')
    plt.plot(np.sqrt(val_errors), 'b-', linewidth = 3, label = 'val')

lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)

In [ ]:
# Code4.14
# 模拟过拟合

from sklearn.pipeline import Pipeline

polynomial_regression = Pipeline([
    ('poly_features', PolynomialFeatures(degree = 10, include_bias = False)),
    ('lin_reg', LinearRegression())
])

plot_learning_curves(polynomial_regression, X, y)

In [ ]:
# Code4.15
# 岭回归

from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha = 1, solver = 'cholesky')
ridge_reg.fit(X, y)
print(ridge_reg.predict([[1.5]]))

sgd_reg = SGDRegressor(penalty = 'l2')
sgd_reg.fit(X, y.ravel())
print(sgd_reg.predict([[1.5]]))

In [ ]:
# Code4.16
# Lasso

from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha = 0.1)
# 也可以使用这个
# lasso_reg = SGDRegressor(penalty = 'l1')

lasso_reg.fit(X, y)
print(lasso_reg.predict([[1.5]]))

In [ ]:
# Code4.17
# 弹性网络

from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha = 0.1, l1_ratio = 0.5)
elastic_net.fit(X, y)
print(elastic_net.predict([[1.5]]))

In [ ]:
# Code4.18
# 2021-01-17 修改
# 提前停止法的基本实现

# 准备数据
np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

# 数据预处理
X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size = 0.5, random_state = 10)

poly_scaler = Pipeline([
    ('poly_features', PolynomialFeatures(degree = 90, include_bias = False)),
    ('std_scaler', StandardScaler())
])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

# 训练与选择模型
from copy import deepcopy

sgd_reg = SGDRegressor(max_iter = 1, tol = -np.infty, warm_start = True, penalty = None, learning_rate = 'constant', eta0 = 0.0005, random_state = 42)

minimum_val_error = float('inf')
best_epoch = None
best_model = None

for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)

print(best_epoch)
print(best_model)

In [ ]:
# Code4.19
# 加载数据集

from sklearn.datasets import load_iris

iris = load_iris()
print(iris.DESCR)

X = iris['data'][:, 3:]
y = (iris['target'] == 2).astype(np.int)

In [ ]:
# Code4.20
# 逻辑回归训练

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X, y)

In [ ]:
# Code4.21
# 修改于2021-01-17
# 逻辑回归测试

X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
decision_boundary = X_new[y_proba[:, 1] >= 0.5][0]

plt.figure(figsize = (8, 3))
plt.plot(X[y == 0], y[y == 0], 'bs')
plt.plot(X[y == 1], y[y == 1], 'g^')
plt.plot([decision_boundary, decision_boundary], [-1, 2], 'k:', linewidth = 2)
plt.plot(X_new, y_proba[:, 1], 'g-', linewidth = 2, label = 'Iris virginica')
plt.plot(X_new, y_proba[:, 0], 'b--', linewidth = 2, label = 'Not Iris virginica')
plt.text(decision_boundary + 0.02, 0.15, 'Decision boundary', fontsize = 14, color = 'k', ha = 'center')
plt.arrow(decision_boundary, 0.08, -0.3, 0, head_width = 0.05, head_length = 0.1, fc = 'b', ec = 'b')
plt.arrow(decision_boundary, 0.92, 0.3, 0, head_width = 0.05, head_length = 0.1, fc = 'g', ec = 'g')
plt.xlabel('Petal width (cm)', fontsize = 14)
plt.ylabel('Probability', fontsize = 14)
plt.legend(loc = 'center left', fontsize = 14)
plt.axis([0, 3, -0.02, 1.02])

plt.show()

In [ ]:
# Code4.22
# Softmax回归

X = iris['data'][:, (2, 3)]
y = iris['target']

softmax_reg = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', C = 10)
softmax_reg.fit(X, y)

In [ ]:
# Code4.23
# Softmax测试

print(softmax_reg.predict([[5,2]]))
print(softmax_reg.predict_proba([[5, 2]]))